<a href="https://colab.research.google.com/github/skabra5/Streaming-tweets-with-Tweepy-Visualing-tweets-using-Plotly-real-time-topic-tracking-with-Dash/blob/master/DashApp2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import os

In [0]:
os.chdir("/content/drive/My Drive/Colab Notebooks/TwitterDash")

In [1]:
!pip install dash dash-renderer dash-html-components dash-core-components plotly

     |████████████████████████████████| 71kB 3.1MB/s 
     |████████████████████████████████| 1.2MB 14.1MB/s 
     |████████████████████████████████| 194kB 44.1MB/s 
     |████████████████████████████████| 3.4MB 32.4MB/s 
     |████████████████████████████████| 1.8MB 49.4MB/s 
     |████████████████████████████████| 358kB 28.6MB/s 
  Created wheel for dash: filename=dash-1.12.0-cp36-none-any.whl size=73892 sha256=87bd261b88b0819cf44c95bc2a19b07eab1b71aea3ece8512d51f7fe0c5d48f1
  Stored in directory: /root/.cache/pip/wheels/2e/bc/dd/b4ebecb434c90eff7c5475efc10b72aad5a899c01fa87e2647
  Created wheel for dash-renderer: filename=dash_renderer-1.4.1-cp36-none-any.whl size=1165597 sha256=0c49f28d0ab0e836ee4cd1065d4815f03df3c3356fad006ed963fc42f825aaa4
  Stored in directory: /root/.cache/pip/wheels/be/9e/83/c6531c3fdd6dba27156e7196c273148d850c97bda396fec7d0
  Created wheel for dash-html-components: filename=dash_html_components-1.0.3-cp36-none-any.whl size=427303 sha256=a5e73910eaca2a5dc5e93f

In [2]:
import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Output, Input, State
import pandas as pd
import plotly.graph_objs as go
from django.conf import settings
import itertools
import math
import base64
from flask import Flask
import os
import psycopg2
import datetime
import plotly
import random
from collections import deque
 
import re
import nltk
nltk.download('punkt')
nltk.download('stopwords')
from nltk.probability import FreqDist
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from textblob import TextBlob
import sqlite3



/usr/local/lib/python3.6/dist-packages/psycopg2/__init__.py:144: UserWarning:

The psycopg2 wheel package will be renamed from release 2.8; in order to keep installing from binary please use "pip install psycopg2-binary" instead. For details see: <http://initd.org/psycopg/docs/install.html#binary-install-from-pypi>.



[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [3]:
### Install ngrok
!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
!unzip ngrok-stable-linux-amd64.zip

### Run ngrok to tunnel Dash app port 8050 to the outside world. 
### This command runs in the background.
get_ipython().system_raw('./ngrok http 8050 &')

### Get the public URL where you can access the Dash app. Copy this URL.
! curl -s http://localhost:4040/api/tunnels | python3 -c \
    "import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])"


--2020-06-10 07:07:44--  https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
Resolving bin.equinox.io (bin.equinox.io)... 52.2.129.46, 34.238.5.126, 3.229.211.71, ...
Connecting to bin.equinox.io (bin.equinox.io)|52.2.129.46|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 13773305 (13M) [application/octet-stream]
Saving to: ‘ngrok-stable-linux-amd64.zip’

ngrok-stable-linux- 100%[===================>]  13.13M  18.5MB/s    in 0.7s    

2020-06-10 07:07:45 (18.5 MB/s) - ‘ngrok-stable-linux-amd64.zip’ saved [13773305/13773305]

Archive:  ngrok-stable-linux-amd64.zip
  inflating: ngrok                   
https://189ca5ca9281.ngrok.io


In [4]:
from collections import deque, Counter
# download nltk dependencies
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('vader_lexicon')

def bag_of_words(series):
    """
    count the words in all the tweets
    Parameters
    ----------
        seriers: pandas Series
            the text column that contains the text of the tweets
    Returns
    -------
        collections.Counter object
            a dictionary with all the tokens and their number of apperances
    """

    # merge the text from all the tweets into one document
    document = ' '.join([row for row in series])

    # lowercasing, tokenization, and keep only alphabetical tokens
    tokens = [word for word in word_tokenize(document.lower()) if word.isalpha()]

    # filtering out tokens that are not all alphabetical
    tokens = [word for word in re.findall(r'[A-Za-z]+', ' '.join(tokens))]

    # remove all stopwords
    # stop words for the word-counts
    stops = stopwords.words('english')
    stops.append('https')
    no_stop = [word for word in tokens if word not in stops]

    return Counter(no_stop)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package vader_lexicon to /root/nltk_data...


In [5]:
%%writefile xapp.py
import os
import dash
import datetime
import dash_core_components as dcc
import dash_html_components as html
# import pandas_datareader.data as web
from dash.dependencies import Output, Input
import plotly
import random
import plotly.graph_objs as go
from collections import deque
import sqlite3
import pandas as pd

#popular topics: google, olympics, trump, gun, usa

# global refresh interval for the application, ms
GRAPH_INTERVAL = os.environ.get("GRAPH_INTERVAL", 2000)

keywords_to_hear = ['#racism',
                    '#blacklivesmatter',
                    '#usa',
                    ]

# global color setting
app_color = {
    "graph_bg": "rgb(221, 236, 255)",
    "graph_line": "rgb(8, 70, 151)",
    "graph_font":"rgb(2, 29, 65)"
}

# colors for plots
chart_colors = [
    '#664DFF',
    '#893BFF',
    '#3CC5E8',
    '#2C93E8',
    '#0BEBDD',
    '#0073FF',
    '#00BDFF',
    '#A5E82C',
    '#FFBD42',
    '#FFCA30'
]

# initialize x and y coordinates for scatter plot
# use duque here to store the changing trend of number of tweets
# X is the x-axis with time stamps
X_universal = deque(maxlen=30)

# the number of most frequently mentioned tags
num_tags_scatter = 5

# initalize a dictionary to store the number of tweets for each game
scatter_dict = {}

sentiment_dict = {}

xapp = dash.Dash(__name__)

xapp.layout = html.Div([
                       html.H2("Real-time-Twitter-Streaming", style={"textAlign": "center"}),
                          html.H5("Last Update: June 08 2020" , style = {"textAlign": "right"}),
                              html.Div(
                                  [dcc.Interval(
                                        id="query_update",
                                        interval=int(GRAPH_INTERVAL),
                                        n_intervals=0,),
                                   html.Div(
                                        [html.H6("WORD-COUNT TREND", className="graph__title")]
                                   ),
                                   html.Div([
                                             html.P(
                                            "Total number of tweets streamed during last 60 seconds: 0",
                                            id="bin-size",
                                            className="auto__p",
                                        ),
                                    ],
                                    className="auto__container",),
                                    dcc.Graph(
                                    id="number_of_tweets", animate=False,
                                    figure=go.Figure(
                                      layout=go.Layout(
                                        plot_bgcolor=app_color["graph_bg"],
                                          paper_bgcolor=app_color["graph_bg"],

                                      )
                                    ),
                                    ),       
                               ],  className="two-thirds column number_of_tweets",
)])
                              

# define callback function for number_of_tweets scatter plot
@xapp.callback(Output('number_of_tweets', 'figure'),[Input('query_update', 'n_intervals')])

def update_graph_scatter(n):
  
  conn = sqlite3.connect('/content/drive/My Drive/Colab Notebooks/TwitterDash/mytwitter.db')

  df = pd.read_sql_query("SELECT * from sentiment", conn)
  c = conn.cursor()

  # # query tweets from the database
  # df = get_tweet_data()

  # get the number of tweets for each keyword
  cnt = bag_of_words(df['tweet'])

  # get the current time for x-axis
  time = datetime.datetime.now().strftime('%D, %H:%M:%S')
  X_universal.append(time)

  to_pop = []
  for keyword, cnt_queue in scatter_dict.items():
      if cnt_queue:
          while cnt_queue and (cnt_queue[0][1] < X_universal[0]):
              cnt_queue.popleft()
      else:
          to_pop.append(keyword)
      

  for keyword in to_pop:
      scatter_dict.pop(keyword)

  top_N = cnt.most_common(num_tags_scatter)

  for keyword, cnt in top_N:
      if keyword not in scatter_dict:
          scatter_dict[keyword] = deque(maxlen=30)
          scatter_dict[keyword].append([cnt, time])
      else:
          scatter_dict[keyword].append([cnt, time])

  new_colors = chart_colors[:len(scatter_dict)]

  # plot the scatter plot
  data=[go.Scatter(
      x=[time for cnt, time in cnt_queue],
      y=[cnt for cnt, time in cnt_queue],
      name=keyword,
      mode='lines+markers',
      opacity=0.5,
      marker=dict(
          size=10,
          color=color,
      ),
      line=dict(
          width=6,
          # dash='dash',
          color=color,
      )
  ) for color, (keyword, cnt_queue) in list(zip(new_colors, scatter_dict.items()))]

  # specify the layout
  layout = go.Layout(
          xaxis={
              'automargin': False,
              'range': [min(X_universal), max(X_universal)],
              'title': 'Current Time (GMT)',
              'nticks': 6
          },
          yaxis={
              'type': 'log',
              'autorange': True,
              'title': 'Number of Tweets'
          },
          height=700,
          plot_bgcolor=app_color["graph_bg"],
          paper_bgcolor=app_color["graph_bg"],
          font={"color": app_color["graph_font"]},
          autosize=False,
          legend={
              'orientation': 'h',
              'xanchor': 'center',
              'yanchor': 'top',
              'x': 0.5,
              'y': 1.025
          },
          margin=go.layout.Margin(
              l=75,
              r=25,
              b=45,
              t=25,
              pad=4
          ),
      )

  # return go.Figure(
  #     data=data,
  #     layout=layout,

  return html.Div(dcc.Graph(id='number_of_tweets', figure=dict(data=data,layout=layout)))
                           
                        
if __name__ == '__main__':
    xapp.run_server(debug=True)

Writing xapp.py


In [6]:
!python xapp.py

Running on http://127.0.0.1:8050/
Debugger PIN: 589-701-693
 * Serving Flask app "xapp" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: on
Running on http://127.0.0.1:8050/
Debugger PIN: 615-570-673
